## Brute Force Evaluation of Search Space
In this notebook, we evaluate the IC50 and LogP of all molecules from a large serach space to provide a baseline for the RL agent.

In [1]:
from molgym.mpnn.layers import custom_objects
from baseline import load_molecules, compute_rdkit, compute_ic50, write_output
import tensorflow as tf
import json
import os

Configuration

In [2]:
search_space = os.path.join('..', '..', 'search-spaces', 'E15.csv')
mpnn_dir = os.path.join('..', 'mpnn-training')

## Define Processing Pipeline
Load, apply transformations, and write back to disk

Load from file as a generator

In [3]:
gen = load_molecules(search_space, chunk_size=1024)

Compute chemical properties with RDKit

In [4]:
gen = compute_rdkit(gen)

Load in the MPNN and components needed to featurize SMILES strings

In [5]:
with open(os.path.join(mpnn_dir, 'atom_types.json')) as fp:
    atom_types = json.load(fp)

In [6]:
with open(os.path.join(mpnn_dir, 'bond_types.json')) as fp:
    bond_types = json.load(fp)

In [7]:
model = tf.keras.models.load_model(os.path.join(mpnn_dir, 'best_model.h5'), custom_objects=custom_objects)

/home/wardlt/miniconda3/envs/covid_dqn-conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/wardlt/miniconda3/envs/covid_dqn-conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/wardlt/miniconda3/envs/covid_dqn-conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/wardlt/miniconda

In [8]:
gen = compute_ic50(gen, model, atom_types, bond_types)

## Run it
It's a big data file to process, so we are going to use the Python functional tools to run it as a stream

In [9]:
write_output(gen, os.path.basename(search_space) + '.gz')

15547090it [7:46:43, 555.19it/s]
